In [ ]:
import torch
from torch import nn
from tests_backpropagation import main_test

torch.manual_seed(123)
torch.set_default_dtype(torch.double)

## Class ``MyNet``

Read carefully how ``MyNet`` is implemented in the cell below. In particular:  
- ``n_hid`` is a list of integer, representing the number of hidden units in each hidden layer.   
-  ``MyNet([2, 3, 2]) = MiniNet()`` where ``MiniNet`` is the neural network defined in the fourth tutorial, in which notations are also clarified.     
- ``model.L`` is the number of hidden layers, ``L``   
- ``model.f[l]`` is the activation function of layer ``l``, $f^{[l]}$ (here ``torch.tanh``)   
- ``model.df[l]`` is the derivative of the activation function, $f'^{[l]}$   
- ``model.a[l]``  is the tensor $A^{[l]}$, (shape: ``(1, n(l))``)   
- ``model.z[l]``  is the tensor $Z^{[l]}$, (shape: ``(1, n(l))``)  
- Weights $W^{[l]}$ (shape: ``(n(l+1), n(l))``) and biases $\mathbf{b}^{[l]}$ (shape: ``(n(l+1))``) can be accessed as follows:
```
weights = model.fc[str(l)].weight.data
bias = model.fc[str(l)].bias.data
```

In [ ]:
class MyNet(nn.Module):
    def __init__(self, n_l = [2, 3, 2]):
        super().__init__() 
        
        
        # number of layers in our network (following Andrew's notations)
        self.L = len(n_l)-1
        self.n_l = n_l
        
        # Where we will store our neuron values
        # - z: before activation function 
        # - a: after activation function (a=f(z))
        self.z = {i : None for i in range(1, self.L+1)}
        self.a = {i : None for i in range(self.L+1)}

        # Where we will store the gradients for our custom backpropagation algo
        self.dL_dw = {i : None for i in range(1, self.L+1)}
        self.dL_db = {i : None for i in range(1, self.L+1)}

        # Our activation functions
        self.f = {i : lambda x : torch.tanh(x) for i in range(1, self.L+1)}

        # Derivatives of our activation functions
        self.df = {i : lambda x : (1 / (torch.cosh(x)**2)) for i in range(1, self.L+1)}
        
        # fully connected layers
        # We have to use nn.ModuleDict and to use strings as keys here to 
        # respect pytorch requirements (otherwise, the model does not learn)
        self.fc = nn.ModuleDict({str(i): None for i in range(1, self.L+1)})
        for i in range(1, self.L+1):
            self.fc[str(i)] = nn.Linear(in_features=n_l[i-1], out_features=n_l[i])
        
    def forward(self, x):
        # Input layer
        self.a[0] = torch.flatten(x, 1)
        
        # Hidden layers until output layer
        for i in range(1, self.L+1):

            # fully connected layer
            self.z[i] = self.fc[str(i)](self.a[i-1])
            # activation
            self.a[i] = self.f[i](self.z[i])

        # return output
        return self.a[self.L]

## Tasks

Write a function ``backpropagation(model, y_true, y_pred)`` that computes:

- $\frac{\partial L}{\partial w^{[l]}_{i,j}}$ and store them in ``model.dL_dw[l][i,j]`` for $l \in [1 .. L]$ 
- $\frac{\partial L}{\partial b^{[l]}_{j}}$ and store them in ``model.dL_db[l][j]`` for $l \in [1 .. L]$ 

assuming ``model`` is an instance of the ``MyNet`` class.

A vectorized implementation would be appreciated.

In [37]:
def backpropagation(model, y_true, y_pred):
    
    with torch.no_grad():
        
        delta = -2 * (y_true - y_pred)

        for l in reversed(range(1, model.L+1)):

            d_f = model.df[l](model.z[l])
            delta = delta * d_f
            
            model.dL_dw[l] = torch.matmul(delta.T, model.a[l-1])
            model.dL_db[l] = delta[0]

            delta = torch.matmul(delta, model.fc[str(l)].weight.data)
    return None

## Run the cells below, and check the output

- In the 1st cell, we use a toy dataset and the same architecture as the MiniNet class of the fourth tutorial. 
- In the 2nd cell, we use a few samples of the MNIST dataset with a consistent model architecture (``24x24`` black and white cropped images as input and ``10`` output classes). 

You can set ``verbose`` to ``True`` if you want more details about your computations versus what is expected.

In [38]:
model = MyNet([2, 3, 2])
main_test(backpropagation, model, verbose=True, data='toy')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

 --------- Comparing with autograd values  ----------- 

 ******* fc['1'].weight.grad ******* 
  Our computation:
 tensor([[ 1.7922e-11,  1.8548e-11],
        [-6.0872e-05, -6.2999e-05],
        [ 1.0689e-09,  1.1062e-09]])

  Autograd's computation:
 tensor([[ 1.7922e-11,  1.8548e-11],
        [-6.0872e-05, -6.2999e-05],
        [ 1.0689e-09,  1.1062e-09]])

 ********* fc['1'].bias.grad ******* 
  Our computation:
 tensor([ 2.1033e-12, -7.1437e-06,  1.2544e-10])
  Autograd's computation:
 tensor([ 2.1032e-12, -7.1437e-06,  1.2544e-10])

 ------------------- relative error ------------------ 
(fc[1]

In [39]:
model = MyNet([24*24, 16, 10])
main_test(backpropagation, model, verbose=True, data='mnist')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 


/Users/emil/opt/anaconda3/envs/inf265/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.0, 0.0, 0.0, 0.0, 0.0]

 --------- Comparing with autograd values  ----------- 

 ******* fc['1'].weight.grad ******* 
  Our computation:
 tensor([[3.3559e-115, 3.3559e-115, 3.3559e-115,  ..., 3.3559e-115,
         3.3559e-115, 3.3559e-115],
        [-1.7124e-34, -1.7124e-34, -1.7124e-34,  ..., -1.7124e-34,
         -1.7124e-34, -1.7124e-34],
        [-1.0897e-99, -1.0897e-99, -1.0897e-99,  ..., -1.0897e-99,
         -1.0897e-99, -1.0897e-99],
        ...,
        [2.9892e-104, 2.9892e-104, 2.9892e-104,  ..., 2.9892e-104,
         2.9892e-104, 2.9892e-104],
        [-6.3804e-61, -6.3804e-61, -6.3804e-61,  ..., -6.3804e-61,
         -6.3804e-61, -6.3804e-61],
        [-4.8096e-10, -4.8096e-10, -4.8096e-10,  ..., -4.8096e-10,
         -4.8096e-10, -4.8096e-10]])

  Autograd's computation:
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,